In [3]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    
print_gpu_utilization()

GPU memory occupied: 16014 MB.


In [2]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer, AutoTokenizer, AutoModelForSeq2SeqLM
from huggingface_hub import notebook_login
import torch
from datasets import load_dataset, load_metric

# Login to huggingface hub
notebook_login()

torch.cuda.empty_cache()
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "google/pegasus-cnn_dailymail"

print("Before start")
print_gpu_utilization()
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
print("After loading tokenizer")
print_gpu_utilization()

def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length = 1024, truncation = True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length = 128, truncation = True)
    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

dataset_samsum = load_dataset("samsum")
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type = "torch", columns = columns)
print("After loading tokenizer")
print_gpu_utilization()

# Now we need to create a data collator. This function is called in the Trainer just before the batch is fed through the model
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)
training_args = TrainingArguments(
    output_dir = 'pegasus-samsum', num_train_epochs = 1, warmup_steps = 500, per_device_train_batch_size = 1, per_device_eval_batch_size = 1,
    weight_decay = 0.01, logging_steps = 10, push_to_hub = True, evaluation_strategy = 'steps', eval_steps = 500, save_steps = 1e6, gradient_accumulation_steps = 16)
trainer = Trainer(model = model, args = training_args, tokenizer = tokenizer, data_collator = seq2seq_data_collator, 
                 train_dataset = dataset_samsum_pt["train"], eval_dataset = dataset_samsum_pt["validation"])
print("Before training")
print_gpu_utilization()
result = trainer.train()
print("Training done...Push to hub")
print_gpu_utilization()
print_summary(result)
trainer.push_to_hub("Training Complete")
#score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = "dialogue", column_summary = "summary")
#rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
#pd.DataFrame(rouge_dict, index = ["pegasus"])

# Finally push to hub
#trainer.push_to_hub("Training Complete")

Before start
GPU memory occupied: 283 MB.
After loading tokenizer
GPU memory occupied: 283 MB.


Reusing dataset samsum (/home/tanvir/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

Upload file pytorch_model.bin:   0%|          | 32.0k/2.13G [00:00<?, ?B/s]

Upload file spiece.model:   2%|1         | 32.0k/1.82M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.92k/2.92k [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/tanviraumi/pegasus-samsum
   e32a1fe..9792bc5  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'dataset': {'name': 'samsum', 'type': 'samsum', 'args': 'samsum'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/tanviraumi/pegasus-samsum
   9792bc5..2001c59  main -> main



'https://huggingface.co/tanviraumi/pegasus-samsum/commit/9792bc5f7875196d8003e395a4b6421fb5a78f08'

In [7]:
from tqdm import tqdm
import torch
from datasets import load_metric
import pandas as pd

rouge_metric = load_metric("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]
        
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size = 16, device = device, column_text = "article", column_summary = "highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))
    
    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total = len(article_batches)):
        inputs = tokenizer(article_batch, max_length = 1024, truncation = True, padding = "max_length", return_tensors = "pt")
        summaries = model.generate(input_ids = inputs["input_ids"].to(device), attention_mask = inputs["attention_mask"].to(device), length_penalty = 0.8, num_beams = 8, max_length = 128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True, clean_up_tokenization_spaces = True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions = decoded_summaries, references = target_batch)
    score = metric.compute()
    return score

score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = "dialogue", column_summary = "summary")
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = ["pegasus"])

100%|████████████████████████████████████████████████████████████████████| 410/410 [05:50<00:00,  1.17it/s]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.398503,0.175022,0.311587,0.311475


In [10]:
# Lets see how our newly trained model performs against a sample
from transformers import pipeline
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization", model = "tanviraumi/pegasus-samsum")

print("Dialogue:")
print(sample_text)
print("\nReference Summary:")
print(reference)
print("\nModel Summary")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

loading configuration file https://huggingface.co/tanviraumi/pegasus-samsum/resolve/main/config.json from cache at /home/tanvir/.cache/huggingface/transformers/6178c373849928347b852c67f88178ee19e69b4e8d2066f00880221675e82ea4.b46a75d1f06c6187e9e1e283bc240896e6772d607bc51a4386380939514cf055
Model config PegasusConfig {
  "_name_or_path": "tanviraumi/pegasus-samsum",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
 

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary
Amanda can't find Betty's number. Larry called Betty last time they were at the park together. He's very nice. Hannah would rather she text him.


In [11]:
custom_conversation = """\
Tanvir: Hi guys, have you heard of Highspot or sales enablement?
Kurt: No, explain it to me
Tanvir: Highspot is a company that provides sales enablement. It helps your sales people to perform sales efficiently.
Kurt: Tell me more
Tanvir: Steve can do it better
Steve: HIghpot does sales enablement by providing a content aggregation platform along with training, coaching, and engagement tools. For sales enablement, Highspot is all you need. 
"""
print(pipe(custom_conversation, **gen_kwargs)[0]["summary_text"])


Your max_length is set to 128, but you input_length is only 96. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)


Highspot is a company that provides sales enablers. It helps your sales people to perform sales efficiently. Kurt can do it better, Steve can do it better.
